In [1]:
library(R2jags)

Loading required package: rjags

Warning message:
"package 'rjags' was built under R version 3.6.3"
Loading required package: coda

Warning message:
"package 'coda' was built under R version 3.6.3"
Linked to JAGS 4.3.0

Loaded modules: basemod,bugs


Attaching package: 'R2jags'


The following object is masked from 'package:coda':

    traceplot




In [3]:
citation('R2jags')


To cite package 'R2jags' in publications use:

  Yu-Sung Su and Masanao Yajima, (2021). R2jags: Using R to Run 'JAGS'.
  R package version 0.7-1. https://CRAN.R-project.org/package=R2jags

A BibTeX entry for LaTeX users is

  @Manual{,
    title = {R2jags: Using R to Run 'JAGS'},
    author = {Yu-Sung Su and Masanao Yajima,},
    year = {2021},
    note = {R package version 0.7-1},
    url = {https://CRAN.R-project.org/package=R2jags},
  }

ATTENTION: This citation information has been auto-generated from the
package DESCRIPTION file and may need manual editing, see
'help("citation")'.


In [7]:
# Define model
model<-function(){

        # priors
        mu_ind0 ~dnorm(0, 0.0001)   # group mean intercept
        mu_ind1 ~dnorm(0, 0.0001)   # group mean slope
        sigma2_ind0 ~dunif(0, 10) # group variance intercept
        sigma2_ind1 ~dunif(0, 10) # group variance slope 
        rho~dunif(-1, 1) # correlationbetweeninterceptsandslopes

        # variance-covariancematrix
        Sigma_B[1, 1] <-sigma2_ind0
        Sigma_B[2, 2] <-sigma2_ind1S
        igma_B[1, 2] <-rho* sigma2_ind0^0.5 * sigma2_ind1^0.5
        Sigma_B[2, 1] <-Sigma_B[1, 2]
        Tau_B[1:2, 1:2] <-inverse(Sigma_B[,])

        cov_ind0_ind1 <-Sigma_B[1, 2]

        # heteroscedasticerrora~dunif(0, 10)   
        b~dnorm(0, 0.0001)

        # autocorrelation
        for(ar in 1:7){phi[ar] ~dunif(-1,1)  }

        for(n in 1:N) {
                B_hat[n, 1] <-mu_ind0
                B_hat[n, 2] <-mu_ind1
                B[n, 1:2] ~dmnorm(B_hat[n, ], Tau_B[,])
                ind0[n] <-B[n, 1] # randomintercept
                ind1[n] <-B[n, 2] # randomslope

                for(i in 1:nobs[n]){
                tau_e[i,n] <-(a*exp(b*x_1[i,n]))^-1   # precision
                trend[i,n] <-ind0[n] + ind1[n]*x_1[i,n]  
                }

                mu_x[2,n] <-trend[2,n] + phi[1]*(y[1,n]-trend[1,n])
                mu_x[3,n] <-trend[3,n]+ phi[1]*(y[2,n]-trend[2,n]) + phi[2]*(y[1,n]-trend[1,n])
                mu_x[4,n] <-trend[4,n] + phi[1]*(y[3,n]-trend[3,n]) + phi[2]*(y[2,n]-trend[2,n]) + phi[3]*(y[1,n]-trend[1,n])
                mu_x[5,n] <-trend[5,n] + phi[1]*(y[4,n]-trend[4,n]) + phi[2]*(y[3,n]-trend[3,n]) + phi[3]*(y[2,n]-trend[2,n]) + phi[4]*(y[1,n]-trend[1,n])
                mu_x[6,n] <-trend[6,n] + phi[1]*(y[5,n]-trend[5,n]) + phi[2]*(y[4,n]-trend[4,n]) + phi[3]*(y[3,n]-trend[3,n]) + phi[4]*(y[2,n]-trend[2,n]) +phi[5]*(y[1,n]-trend[1,n])
                mu_x[7,n] <-trend[7,n] + phi[1]*(y[6,n]-trend[6,n]) + phi[2]*(y[5,n]-trend[5,n]) + phi[3]*(y[4,n]-trend[4,n]) + phi[4]*(y[3,n]-trend[3,n]) + phi[5]*(y[2,n]-trend[2,n]) + phi[6]*(y[1,n]-trend[1,n])

                # likelihood
                y[2,n] ~dnorm(mu_x[2,n], tau_e[2,n])
                y[3,n] ~dnorm(mu_x[3,n], tau_e[3,n])
                y[4,n] ~dnorm(mu_x[4,n], tau_e[4,n])
                y[5,n] ~dnorm(mu_x[5,n], tau_e[5,n])
                y[6,n] ~dnorm(mu_x[6,n], tau_e[6,n])
                y[7,n] ~dnorm(mu_x[7,n], tau_e[7,n])

                for(i in 8:nobs[n]){
                        mu_x[i,n] <-trend[i,n] +
                        phi[1]*(y[i-1,n]-trend[i-1,n]) +
                        phi[2]*(y[i-2,n]-trend[i-2,n]) +
                        phi[3]*(y[i-3,n]-trend[i-3,n]) + 
                        phi[4]*(y[i-4,n]-trend[i-4,n]) + 
                        phi[5]*(y[i-5,n]-trend[i-5,n]) +
                        phi[6]*(y[i-6,n]-trend[i-6,n]) + 
                        phi[7]*(y[i-7,n]-trend[i-7,n])
                        y[i,n] ~dnorm(mu_x[i,n], tau_e[i,n])         
                        }
        }
}

In [ ]:

# initialvalues
init<-list(mu_ind0=1, mu_ind1=1, sigma2_ind0=0.1, sigma2_ind1=0.1, phi=rep(0,7), rho=0, a=1, b=0)
inits<-list(init, init, init)

# parameterstosave
pars<-c('sigma2_ind0','sigma2_ind1', 'cov_ind0_ind1', 'a','b','phi')

# data
load('daily.activity.Rdata')
total.dist.raw.dat<-list(y=total.dist, x_1=temp, N=N, nobs=nobs)

# run model
model<-jags(data=total.dist.raw.dat, parameters.to.save=pars,model.file=model, n.chains=3, n.iter=600000, n.burnin=100000, n.thin=100)